In [2]:
import math
import numpy as np
import pandas as pd
validator = "Coinbase_processed.csv"
path = f"./{validator}"
df_data= pd.read_csv(path)

In [3]:
df_data['block_len'][1:].mean()

6.67192189218922

In [4]:
# df_data

In [5]:
total_missed = 0
indices = df_data['block_num']
for i in range(1,len(indices)):
    if int(indices[i]) != int(indices[i-1])+1:
        total_missed += 1
        print(i)

In [6]:
print(total_missed)
print(len(df_data))

0
304


In [7]:
df_data.reset_index(inplace=True)

In [8]:
print(df_data)

     index  block_num  Block_length_target  Goal_Bonded  \
0        0   12788312             7.233028       0.6666   
1        1   12788313             7.233028       0.6666   
2        2   12788314             7.233028       0.6666   
3        3   12788315             7.233028       0.6666   
4        4   12788316             7.233028       0.6666   
..     ...        ...                  ...          ...   
299    299   12788611             7.233028       0.6666   
300    300   12788612             7.233028       0.6666   
301    301   12788613             7.233028       0.6666   
302    302   12788614             7.233028       0.6666   
303    303   12788615             7.233028       0.6666   

     Inflation_Rate_Change  Min_Inflation_Rate  Max_Inflation_Rate  \
0                     0.13                0.07                 0.2   
1                     0.13                0.07                 0.2   
2                     0.13                0.07                 0.2   
3          

In [9]:
df_output = pd.DataFrame()
df_output['block Number'] = df_data.block_num
df_output['Atom Staked'] = df_data.atom_staked_v
df_output['Total_Rewards'] = ( df_data.commission_amt/(df_data.v_commission/100)) * 1E-6
df_output['Epoch Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Expected Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Realized APY'] = df_output.apply(lambda _: '', axis=1)
df_output['Expected APY'] = df_output.apply(lambda _: '', axis=1)
df_output['Absolute Difference in Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Absolute Difference in APY'] = df_output.apply(lambda _: '', axis=1)
df_output['% Difference in Rewards'] = df_output.apply(lambda _: '', axis=1)

print(df_output.head())

   block Number   Atom Staked  Total_Rewards Epoch Rewards Expected Rewards  \
0      12788312  1.565814e+07    4930.746234                                  
1      12788313  1.565814e+07    4931.428396                                  
2      12788314  1.565814e+07    4932.109814                                  
3      12788315  1.565814e+07    4932.792476                                  
4      12788316  1.565814e+07    4934.156362                                  

  Realized APY Expected APY Absolute Difference in Rewards  \
0                                                            
1                                                            
2                                                            
3                                                            
4                                                            

  Absolute Difference in APY % Difference in Rewards  
0                                                     
1                                         

In [10]:
# Constants from Network


# Parameters to be varied by validator
for i in range(1,len(df_data)):

  Goal_Bonded                 = float(df_data.Goal_Bonded[i]) 
  Inflation_Rate_Change       = float(df_data.Inflation_Rate_Change[i]) 
  Min_Inflation_Rate          = float(df_data.Min_Inflation_Rate[i]) 
  Max_Inflation_Rate          = float(df_data.Max_Inflation_Rate[i]) 
  Block_length                = float(df_data.block_len[i]) 
  E_Block_length              = float(df_data.Block_length_target[i])
  Blocks_per_year             = float(df_data.Blocks_per_year[i]) 
  minProposerBonus            = float(df_data.min_proposer_bonus[i]) 
  maxProposerBonus            = float(df_data.max_proposer_bonus[i]) 
  Community_Tax               = float(df_data.community_tax[i]) 
    
  Min_Signatures              = float(df_data.Min_Signatures[i]) 
  Total_Fees_Per_Block        = float(df_data.total_block_fees[i]) * 1E-6
  Bonded_Ratio                = float(df_data.percent_staked[i]) /100
  stake                       = float(df_data.atom_staked_v[i]) 
  signatures_ratio            = float(df_data.sign_ratio[i]) 
  E_Inflation_Rate_BH         = float(df_data.inflation_rate[i]) 
  totalSupply                 = float(df_data.total_supply[i]) * 1E-6

  # E_current_Inflation_rate_change = (1 - Bonded_Ratio/Goal_Bonded) * Inflation_Rate_Change
  # E_Inflation_Rate_BH             = Prev_Inflation_Rate + E_current_Inflation_rate_change / Blocks_per_year

    # Check the new inflation rate is between min and max inflation rate
  if E_Inflation_Rate_BH > Max_Inflation_Rate:
    E_Inflation_Rate_BH = Max_Inflation_Rate
  elif E_Inflation_Rate_BH < Min_Inflation_Rate:
    E_Inflation_Rate_BH = Min_Inflation_Rate

  # Real_Inflation_Rate = E_Inflation_Rate_BH * Block_length/ E_Block_length

  E_validator         = stake / (totalSupply*Bonded_Ratio) * ((1+E_Inflation_Rate_BH)**(1/Blocks_per_year)-1) * totalSupply    # technically can cancel totalSupply but left for ease of understanding
  #E_validator         = stake / (totalSupply*Bonded_Ratio) * E_Inflation_Rate_BH/Blocks_per_year * totalSupply
  E_Proposer_Bonus                     = minProposerBonus + (signatures_ratio-Min_Signatures) / (1-Min_Signatures) * (maxProposerBonus-minProposerBonus)      
  E_Proposer_stake                     = totalSupply*Bonded_Ratio/175   # THIS IS POOR ESTIMATE AS VALIDATORS WITH MORE STAKE ARE SELECTED MORE FREQUENTLY, WOULD NEED ALL VALIDATOR'S DATA TO IMPROVE MODEL
  E_transaction_reward_per_atom_staked = Total_Fees_Per_Block * (1 - Community_Tax) / (totalSupply*Bonded_Ratio + E_Proposer_stake*E_Proposer_Bonus)

  E_ProposerTxFees                     = stake * (1 + E_Proposer_Bonus) * E_transaction_reward_per_atom_staked
  E_AttesterTxFees                     = stake * E_transaction_reward_per_atom_staked

  E_Attester = E_validator + E_AttesterTxFees
  E_Proposer = E_validator + E_ProposerTxFees

  P_Proposer = stake/totalSupply    # think this is true even with round robin selection
  P_attester = 1 - P_Proposer


  E_rewards = E_Proposer*P_Proposer + E_Attester*P_attester
  df_output['Epoch Rewards'][i]       = df_output.Total_Rewards[i]-df_output.Total_Rewards[i-1]
  df_output['Expected Rewards'][i] = E_rewards
  df_output['Absolute Difference in Rewards'][i] = df_output['Expected Rewards'][i] - df_output['Epoch Rewards'][i]
  df_output['% Difference in Rewards'][i] = (df_output['Expected Rewards'][i] - df_output['Epoch Rewards'][i]) / df_output['Epoch Rewards'][i]*100

  APY                                            = E_rewards/stake*Blocks_per_year*E_Block_length/Block_length
  Realized_APY                                   = df_output['Epoch Rewards'][i]/stake*Blocks_per_year*E_Block_length/Block_length
  df_output['Expected APY'][i]                   = APY*100
  df_output['Realized APY'][i]                   = Realized_APY*100     
  df_output['Absolute Difference in Rewards'][i] = E_rewards - df_output['Epoch Rewards'][i]
  df_output['Absolute Difference in APY'][i]     = (df_output['Expected APY'][i]  -   df_output['Realized APY'][i])

print(df_output)

     block Number   Atom Staked  Total_Rewards Epoch Rewards Expected Rewards  \
0        12788312  1.565814e+07    4930.746234                                  
1        12788313  1.565814e+07    4931.428396      0.682163         0.688048   
2        12788314  1.565814e+07    4932.109814      0.681417         0.688048   
3        12788315  1.565814e+07    4932.792476      0.682663         0.688048   
4        12788316  1.565814e+07    4934.156362      1.363886         0.688048   
..            ...           ...            ...           ...              ...   
299      12788611  1.566094e+07    5146.074784      0.682107         0.688183   
300      12788612  1.566094e+07    5147.439051      1.364267         0.688184   
301      12788613  1.566094e+07    5147.439051           0.0         0.688184   
302      12788614  1.566094e+07    5148.120196      0.681144         0.688184   
303      12788615  1.566094e+07    5148.801340      0.681144         0.688184   

    Realized APY Expected A

/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_15269/160660359.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output['Epoch Rewards'][i]       = df_output.Total_Rewards[i]-df_output.Total_Rewards[i-1]
/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_15269/160660359.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output['Expected Rewards'][i] = E_rewards
/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_15269/160660359.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [11]:
# df_output.to_csv(f"drive/Shared drives/Model/Data Extraction/COSMOS/single-validators/Output Files/COSMOSValidationOutput-{validator}")

In [12]:
total_rewards = df_output['Epoch Rewards'][1:].sum()
expected_total = df_output['Expected Rewards'][1:].sum()

In [13]:
acual_total_rew_without_processing = (df_data.iloc[len(df_data)-1]['commission_amt']-df_data.iloc[0]['commission_amt'])/1e6/0.05
(acual_total_rew_without_processing-expected_total)/expected_total*100

322.55239971224967

In [14]:
expected_total

206.41710367990754

In [15]:
acual_total_rew_without_processing

872.2204250159717

In [16]:
(total_rewards-expected_total)/total_rewards*100

5.337184152215619

In [17]:
df_output[1:][df_output['Epoch Rewards'][1:]>0.010]

,block Number,Atom Staked,Total_Rewards,Epoch Rewards,Expected Rewards,Realized APY,Expected APY,Absolute Difference in Rewards,Absolute Difference in APY,% Difference in Rewards
1,12788313,1.565814e+07,4931.428396,0.682163,0.688048,20.766289,20.945438,0.005885,0.179149,0.862693
2,12788314,1.565814e+07,4932.109814,0.681417,0.688048,20.185271,20.381678,0.00663,0.196407,0.973023
3,12788315,1.565814e+07,4932.792476,0.682663,0.688048,20.885687,21.050439,0.005385,0.164752,0.788826
4,12788316,1.565814e+07,4934.156362,1.363886,0.688048,41.544326,20.958115,-0.675838,-20.586211,-49.552401
6,12788318,1.565814e+07,4934.837637,0.681275,0.688048,58.362783,58.94303,0.006773,0.580247,0.994208
...,...,...,...,...,...,...,...,...,...,...
297,12788609,1.566094e+07,5145.392678,1.841177,0.688183,57.695697,21.565133,-1.152993,-36.130564,-62.622632
299,12788611,1.566094e+07,5146.074784,0.682107,0.688183,21.757315,21.951141,0.006077,0.193826,0.890857
300,12788612,1.566094e+07,5147.439051,1.364267,0.688184,43.758933,22.073522,-0.676083,-21.685411,-49.556535
302,12788614,1.566094e+07,5148.120196,0.681144,0.688184,21.826877,22.052447,0.007039,0.225571,1.033454


In [18]:
df_output

,block Number,Atom Staked,Total_Rewards,Epoch Rewards,Expected Rewards,Realized APY,Expected APY,Absolute Difference in Rewards,Absolute Difference in APY,% Difference in Rewards
0,12788312,1.565814e+07,4930.746234,,,,,,,
1,12788313,1.565814e+07,4931.428396,0.682163,0.688048,20.766289,20.945438,0.005885,0.179149,0.862693
2,12788314,1.565814e+07,4932.109814,0.681417,0.688048,20.185271,20.381678,0.00663,0.196407,0.973023
3,12788315,1.565814e+07,4932.792476,0.682663,0.688048,20.885687,21.050439,0.005385,0.164752,0.788826
4,12788316,1.565814e+07,4934.156362,1.363886,0.688048,41.544326,20.958115,-0.675838,-20.586211,-49.552401
...,...,...,...,...,...,...,...,...,...,...
299,12788611,1.566094e+07,5146.074784,0.682107,0.688183,21.757315,21.951141,0.006077,0.193826,0.890857
300,12788612,1.566094e+07,5147.439051,1.364267,0.688184,43.758933,22.073522,-0.676083,-21.685411,-49.556535
301,12788613,1.566094e+07,5147.439051,0.0,0.688184,0.0,21.857661,0.688184,21.857661,inf
302,12788614,1.566094e+07,5148.120196,0.681144,0.688184,21.826877,22.052447,0.007039,0.225571,1.033454
